In [50]:
# Importing libraries
import pandas as pd

In [51]:
# Loading dataset
file_path = "../data/raw/Online Retail.xlsx"
df = pd.read_excel(file_path)

In [52]:
# Basic inspection
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [53]:
df.shape

(541909, 8)

In [54]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [55]:
# Standardise column names
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df.columns

Index(['invoiceno', 'stockcode', 'description', 'quantity', 'invoicedate',
       'unitprice', 'customerid', 'country'],
      dtype='object')

## Data Cleaning Approach

The raw dataset contains missing customer identifiers, negative quantities representing returns, and inconsistent formatting. 
This notebook applies defined business rules to prepare a clean, analytics-ready dataset suitable for KPI reporting and dashboarding.


In [56]:
df.isnull().sum().sort_values(ascending=False)

customerid     135080
description      1454
invoiceno           0
stockcode           0
quantity            0
invoicedate         0
unitprice           0
country             0
dtype: int64

In [57]:
df.head()


,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [58]:
# Removed empty customer id rows to make sure our customer analysis is valid
df = df.dropna(subset=["customerid"])
df.shape

(406829, 8)

In [59]:
# Removed negative unit prices it wouldnt affect our revenue KPIs
df = df[df["unitprice"] > 0]
df.shape

(406789, 8)

In [60]:
# Removing negative quantities that represent returns/collections keeping only valid sales transactoins for analysis
df = df[df["quantity"] > 0]
df.shape

(397884, 8)

In [61]:
# Converting invoice date to datetime format enabling us to have time based KPIs
df["invoicedate"] = pd.to_datetime(df["invoicedate"])

In [62]:
df["revenue"] = df["quantity"] * df["unitprice"]

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397884 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoiceno    397884 non-null  object        
 1   stockcode    397884 non-null  object        
 2   description  397884 non-null  object        
 3   quantity     397884 non-null  int64         
 4   invoicedate  397884 non-null  datetime64[ns]
 5   unitprice    397884 non-null  float64       
 6   customerid   397884 non-null  float64       
 7   country      397884 non-null  object        
 8   revenue      397884 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 30.4+ MB


In [68]:
df.describe()

,quantity,invoicedate,unitprice,customerid,revenue
count,397884.000000,397884,397884.000000,397884.000000,397884.000000
mean,12.988238,2011-07-10 23:41:23.511023360,3.116488,15294.423453,22.397000
min,1.000000,2010-12-01 08:26:00,0.001000,12346.000000,0.001000
25%,2.000000,2011-04-07 11:12:00,1.250000,13969.000000,4.680000
50%,6.000000,2011-07-31 14:39:00,1.950000,15159.000000,11.800000
75%,12.000000,2011-10-20 14:33:00,3.750000,16795.000000,19.800000
max,80995.000000,2011-12-09 12:50:00,8142.750000,18287.000000,168469.600000
std,179.331775,NaN,22.097877,1713.141560,309.071041


In [69]:
df.shape

(397884, 9)

In [70]:
df.to_csv("../data/processed/clean_retail_data.csv", index = False)